## Importations

In [2]:
import pandas as pd
import plotly.express as px
file_path = '/Users/benjamincordebar/Desktop/2A/S8/VBD/viz_project/data/chocolate.csv'
data = pd.read_csv(file_path)   # return a dataframe

Afficher le nombre de lignes avant suppression
Supprimer les lignes où la colonne 'ingredients' contient des valeurs manquantes
Afficher le nombre de lignes après suppression
À ce stade, data_cleaned contient votre jeu de données sans les lignes manquantes dans 'ingredients'


In [3]:
print(f"Nombre de lignes avant suppression: {data.shape[0]}")
data_cleaned = data.dropna(subset=['ingredients'])
print(f"Nombre de lignes après suppression: {data_cleaned.shape[0]}")


Nombre de lignes avant suppression: 2530
Nombre de lignes après suppression: 2443


## -- Visualisation exemple : evolution of the notes according to the years -- ###

In [4]:
# Grouper les données par 'review_date' et calculer la moyenne des notes
data_grouped = data_cleaned.groupby('review_date')['rating'].mean().reset_index()

# Créer un graphique linéaire avec Plotly
fig = px.line(data_grouped, x='review_date', y='rating', title='Évolution moyenne des notes par année',
              labels={'review_date': 'Année', 'rating': 'Note moyenne'},color_discrete_sequence=['#2b0808'])

fig.update_layout(plot_bgcolor='#f6eddb', paper_bgcolor='lightgray')

# Afficher le graphique
fig.show()

# Enregistrer le graphique dans un fichier HTML
file_name = '/Users/benjamincordebar/Desktop/2A/S8/VBD/viz_project/dynamic_representations/evolution_of_the_notes_according_to_the_years.html'
fig.write_html(file_name)

## Transformation des données "ingrédients" en données binaire afin de pouvoir de pouvoir utiliser ces données dans un corélation. 

In [5]:
# Décoder les ingrédients
ingredients = {
    'B': 'Beans',
    'S': 'Sugar',
    'S*': 'Sweetener other than sugar',
    'C': 'Cocoa Butter',
    'V': 'Vanilla',
    'L': 'Lecithin',
    'Sa': 'Salt'
}

# Créer une copie indépendante pour éviter les avertissements lors de la modification
data_cleaned2 = data.dropna(subset=['ingredients']).copy()

for ingredient, full_name in ingredients.items():
    data_cleaned2[full_name] = data_cleaned2['ingredients'].apply(lambda x: 1 if ingredient in x else 0)

# Afficher les premières lignes du DataFrame nettoyé avec les nouvelles colonnes d'ingrédients
#print(data_cleaned2.head())


# Grouper les données par 'review_date' et calculer la moyenne des notes et la somme des ingrédients
data_grouped = data_cleaned2.groupby('review_date').agg(
    {'rating': 'mean', 'Beans': 'sum', 'Sugar': 'sum', 'Sweetener other than sugar': 'sum', 
     'Cocoa Butter': 'sum', 'Vanilla': 'sum', 'Lecithin': 'sum', 'Salt': 'sum'}).reset_index()

print(data_grouped.head())


   review_date    rating  Beans  Sugar  Sweetener other than sugar  \
0         2006  3.062500     60     60                           0   
1         2007  3.166667     69     69                           0   
2         2008  3.038690     84     83                           0   
3         2009  3.088496    113    113                           0   
4         2010  3.188830     94     94                           1   

   Cocoa Butter  Vanilla  Lecithin  Salt  
0            56       43        43     0  
1            56       40        28     0  
2            69       33        47     0  
3            95       40        42     0  
4            78       25        43     1  


### Afficher le nombre d'ingrédients en fonction des années.

In [6]:
# Ajouter une colonne pour le total des ingrédients par année
data_grouped['Total_Ingredients'] = data_grouped[['Beans', 'Sugar', 'Sweetener other than sugar', 'Cocoa Butter', 'Vanilla', 'Lecithin', 'Salt']].sum(axis=1)


# Afficher le DataFrame mis à jour
print(data_grouped[['review_date', 'Total_Ingredients']])





    review_date  Total_Ingredients
0          2006                262
1          2007                262
2          2008                316
3          2009                403
4          2010                336
5          2011                483
6          2012                587
7          2013                590
8          2014                710
9          2015                776
10         2016                595
11         2017                284
12         2018                644
13         2019                567
14         2020                213
15         2021                478


In [7]:
# Créer un graphique linéaire pour visualiser l'évolution du nombre total d'ingrédients par année
fig = px.line(data_grouped, x='review_date', y='Total_Ingredients', title='Évolution du nombre total d\'ingrédients par année',
              labels={'review_date': 'Année', 'Total Ingredients': 'Nombre total d\'ingrédients'},color_discrete_sequence=['#2b0808'])

fig.update_layout(plot_bgcolor='#f6eddb', paper_bgcolor='lightgray')

# Afficher le graphique
fig.show()

# Enregistrer le graphique dans un fichier HTML
file_name = '/Users/benjamincordebar/Desktop/2A/S8/VBD/viz_project/dynamic_representations/change_in_total_number_of_ingredients_per_year.html'
fig.write_html(file_name)

In [8]:
print(data_grouped.head())

fig = px.scatter(data_grouped, x="review_date", y="rating", color="Total_Ingredients", 
                 marginal_y="violin", marginal_x="box", trendline="ols", template="simple_white")
fig.show()


   review_date    rating  Beans  Sugar  Sweetener other than sugar  \
0         2006  3.062500     60     60                           0   
1         2007  3.166667     69     69                           0   
2         2008  3.038690     84     83                           0   
3         2009  3.088496    113    113                           0   
4         2010  3.188830     94     94                           1   

   Cocoa Butter  Vanilla  Lecithin  Salt  Total_Ingredients  
0            56       43        43     0                262  
1            56       40        28     0                262  
2            69       33        47     0                316  
3            95       40        42     0                403  
4            78       25        43     1                336  


Modification de la dataframe avec avec de quatégories de nombre d'ingrédients.

In [9]:
# Définir les bins et les labels pour les catégories
bins = [0, 266, 533, 799]  
labels = ['0-266', '267-533', '534-799']

# Créer une nouvelle colonne 'Ingredients_Category'
data_grouped['Ingredients_Category'] = pd.cut(data_grouped['Total_Ingredients'], bins=bins, labels=labels, right=False)

# Afficher le DataFrame mis à jour pour vérifier la nouvelle colonne
print(data_grouped[['review_date', 'Total_Ingredients', 'Ingredients_Category']])

# Créer un graphique linéaire pour visualiser l'évolution des notes moyennes dans le temps par catégorie d'ingrédients
fig = px.scatter(data_grouped, x="review_date", y="rating", color="Ingredients_Category", 
                 marginal_y="violin", marginal_x="box", trendline="ols", template="simple_white")
fig.show()


    review_date  Total_Ingredients Ingredients_Category
0          2006                262                0-266
1          2007                262                0-266
2          2008                316              267-533
3          2009                403              267-533
4          2010                336              267-533
5          2011                483              267-533
6          2012                587              534-799
7          2013                590              534-799
8          2014                710              534-799
9          2015                776              534-799
10         2016                595              534-799
11         2017                284              267-533
12         2018                644              534-799
13         2019                567              534-799
14         2020                213                0-266
15         2021                478              267-533
